In [23]:
import os
#import cv2.cv2 as cv2
import torch
import numpy as np
#from dataset.dataset import MyDataSet

from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.nn import BCELoss
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import transforms

from net.nets import UNet
from setting.setting import best_params_path, last_params_path, train_log_dir, is_save_img
from setting.setting import img_size


import matplotlib.pyplot as plt
class MyDataSet(Dataset):
    def __init__(self, data_dir=voc_data_dir):
        self.train_dir = os.path.join(data_dir, 'JPEGImages')
        self.target_dir = os.path.join(data_dir, 'SegmentationClass')
        self.dataset = os.listdir(self.target_dir)
        print('see',self.dataset[0])
        self.trans = transforms.ToTensor()

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, aaa):
        img_name = self.dataset[aaa]
#        print(img_name)
#        print(img_name[0:-2])
        train_img = Image.open(os.path.join(self.train_dir, f'{img_name[:-3]}jpg'))  # 训练图片
#        print(train_img)
        target_img = Image.open(os.path.join(self.target_dir, img_name))  # 分割标签
#        print(target_img)
        """ 统一尺寸 """
        w, h = train_img.size
#        print(w,h)
        max_border = max(w, h)
#        print(max_border)
        train_img = train_img.resize((int(img_size*w/max_border), int(img_size*h/max_border)))
#        print(train_img.size)
#        print(train_img)
#        print(img_size,w,max_border)
#        print(img_size*w/max_border)
        target_img = target_img.resize((int(img_size*w/max_border), int(img_size*h/max_border)))

        """ 创建数据和标签 """
        train_img_back = transforms.ToPILImage()(torch.zeros((3, img_size, img_size)))
        target_img_back = transforms.ToPILImage()(torch.zeros((3, img_size, img_size)))
        train_img_back.paste(train_img)
        target_img_back.paste(target_img)
#        plt.imshow(train_img_back)
#        plt.show()
#        plt.imshow(target_img_back)
#        plt.show()
        return self.trans(train_img_back), self.trans(target_img_back)


#if __name__ == '__main__':
#    train_dataset = MyDataSet(r'/home/l/20211218 practice/data/unet4/unet_voc-master/dataset/VOCdevkit/VOC2007')
#    print(train_dataset[0])
#    print(len(train_dataset))

ModuleNotFoundError: No module named 'matlab'

In [24]:
batch_size = 12
lr = 0.0001

if __name__ == '__main__':
    summary_writer = SummaryWriter(log_dir=train_log_dir)
    train_dataset = MyDataSet()
    train_dataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    model = UNet().cuda()
#    if os.path.exists(best_params_path):
#        model.load_state_dict(torch.load(best_params_path))
#        print('init params load success')
#    else:
#        print('no params to init')
    loss_fun = BCELoss()
    opt = Adam(model.parameters(), lr=lr)

    epoch = 0
    global_step = 0
    best_params = None
    while True:
        epoch += 1
        epoch_loss = []

        model.train()
        for batch_index, (x, y) in enumerate(train_dataLoader):
            batch_index += 1
            global_step += 1
            x, y = x.cuda(), y.cuda()
        #    print('shape x',x.shape)
        #    print('shape y',y.shape)
            y_pre = model(x)
        #    print('shape y_pre',y_pre.shape)
            loss = loss_fun(y_pre, y)
            #print(loss)
            opt.zero_grad()
            loss.backward()
            opt.step()

            print(f'epoch: {epoch}/{batch_index}  loss ==> {loss.item()}')
            summary_writer.add_scalar('loss', loss.item(), global_step=global_step)
            epoch_loss.append(loss.item())

            if is_save_img:
                """ 保存图片查看效果 """
                img_list = []
                
                for i in range(x.shape[0]):
                    
                    img_list.append(torch.stack((x[i], y[i], y_pre[i]), dim=0))
                    
                img_list = torch.cat(img_list, dim=0)
      #          print('img_list shape',img_list.shape)
                img_save_dir = f'img/epoch={epoch}'
                if not os.path.exists(img_save_dir):
                    os.makedirs(img_save_dir)
                save_image(img_list, os.path.join(img_save_dir, f'{batch_index}.png'))

        """ 模型保存 """
        if epoch == 1:
            best_params = np.mean(epoch_loss)
        elif np.mean(epoch_loss) < best_params:
            best_params = np.mean(epoch_loss)
            torch.save(model.state_dict(), best_params_path)
            print('best model save success')
        
        ############need to be debug
        
        #torch.save(model.state_dict(), last_params_path)

see 009841.png


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [22]:
import os


BASEDIR = r'/home/l/20211218 practice/data/20220528_mask_rCNN/unet_voc-master/dataset/VOCdevkit/VOC2007'
voc_data_dir = r'/home/l/20211218 practice/data/20220528_mask_rCNN/unet_voc-master/dataset/VOCdevkit/VOC2007'

img_size = 224

best_params_path = os.path.join(BASEDIR, 'static_resources/save_model/best_params')
#print(best_params_path)
last_params_path = os.path.join(BASEDIR, 'static_resources/save_model/last_params')
train_log_dir = os.path.join(BASEDIR, 'static_resources/tensorboard_log/train_log')
print(train_log_dir)
is_save_img = True  # 是否保存图片进行查看

/home/l/20211218 practice/data/20220528_mask_rCNN/unet_voc-master/dataset/VOCdevkit/VOC2007/static_resources/tensorboard_log/train_log
